Train a simple binarized Convolutional Neural Network to classify MNIST digits

In [ ]:
pip install larq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import larq as lq

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")

model = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
model.add(lq.layers.QuantConv2D(32, (3, 3),
                                kernel_quantizer="ste_sign",
                                kernel_constraint="weight_clip",
                                use_bias=False,
                                input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())

model.add(lq.layers.QuantDense(64, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantDense(10, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

In [ ]:
lq.models.summary(model)

+sequential stats------------------------------------------------------------------------------------------+
| Layer                  Input prec.           Outputs  # 1-bit  # 32-bit  Memory  1-bit MACs  32-bit MACs |
|                              (bit)                        x 1       x 1    (kB)                          |
+----------------------------------------------------------------------------------------------------------+
| quant_conv2d                     -  (-1, 26, 26, 32)      288         0    0.04           0       194688 |
| max_pooling2d                    -  (-1, 13, 13, 32)        0         0       0           0            0 |
| batch_normalization              -  (-1, 13, 13, 32)        0        64    0.25           0            0 |
| quant_conv2d_1                   1  (-1, 11, 11, 64)    18432         0    2.25     2230272            0 |
| max_pooling2d_1                  -    (-1, 5, 5, 64)        0         0       0           0            0 |
| batch_normalizati

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=64, epochs=6)

test_loss, test_acc = model.evaluate(test_images, test_labels)

Epoch 1/6
938/938 [==============================] - 66s 68ms/step - loss: 0.6500 - accuracy: 0.9072
Epoch 2/6
938/938 [==============================] - 56s 59ms/step - loss: 0.4753 - accuracy: 0.9601
Epoch 3/6
938/938 [==============================] - 56s 59ms/step - loss: 0.4498 - accuracy: 0.9695
Epoch 4/6
938/938 [==============================] - 58s 62ms/step - loss: 0.4364 - accuracy: 0.9726
Epoch 5/6
938/938 [==============================] - 56s 59ms/step - loss: 0.4304 - accuracy: 0.9747
Epoch 6/6
313/313 [==============================] - 5s 14ms/step - loss: 0.5159 - accuracy: 0.9618


In [ ]:
print(f"Test accuracy {test_acc * 100:.2f} %")

Test accuracy 96.18 %


In [ ]:
predictions= model.predict(test_images)

313/313 [==============================] - 3s 9ms/step


In [ ]:
print(predictions[0])

[0.03467151 0.02637186 0.02903052 0.0492917  0.02694386 0.02578786
 0.0350228  0.66962457 0.04895309 0.05430223]


In [ ]:
print(sum(predictions[1]))

1.0000000335276127


In [ ]:
print(len(predictions))

10000


In [ ]:
for y in range (len(predictions)):
  for x in range (len(predictions[y])):
    if(predictions[y][x]==max(predictions[y])):
      print(f"Image number {y} contains the digit {x}")

Streaming output truncated to the last 5000 lines.
Image number 5000 contains the digit 3
Image number 5001 contains the digit 9
Image number 5002 contains the digit 9
Image number 5003 contains the digit 8
Image number 5004 contains the digit 4
Image number 5005 contains the digit 1
Image number 5006 contains the digit 0
Image number 5007 contains the digit 6
Image number 5008 contains the digit 0
Image number 5009 contains the digit 9
Image number 5010 contains the digit 6
Image number 5011 contains the digit 8
Image number 5012 contains the digit 6
Image number 5013 contains the digit 1
Image number 5014 contains the digit 1
Image number 5015 contains the digit 9
Image number 5016 contains the digit 8
Image number 5017 contains the digit 9
Image number 5018 contains the digit 2
Image number 5019 contains the digit 3
Image number 5020 contains the digit 5
Image number 5021 contains the digit 5
Image number 5022 contains the digit 9
Image number 5023 contains the digit 4
Image number 